In [13]:
from datasets import load_from_disk
from datasets import Dataset, DatasetDict

## Opening Dataset + Prompt

In [36]:
def generate_training_prompt_opening(regulatory: str, text: str, triples: str) -> str:
    return f"""
Diberikan sebuah prompt yang dibatasi oleh tiga backticks, lakukan tindakan berikut:
- identifikasi sebanyak mungkin relation di antara entity
- keluarkan daftar dalam format triple turtle ENTITY 1 RELATION 1 OBJECT 1/ENTITY 2 .

Tipe entity paling penting adalah:
- lexid-s:LegalDocument, contoh: lexid:Permen_Agama_2019_17
- lexid-s:Position, contoh: lexid:Position_Menteri_Agama_Republik_Indonesia

Relation paling penting adalah:
- lexid-s:regulationNumber, yaitu nomor dari peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:regulationNumber "12"^^xsd:string .
- lexid-s:regulationYear, yaitu tahun dari peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:regulationNumber "2019"^^xsd:string .
- lexid-s:considers, yaitu apa pertimbangan dari peraturan tersebut (ada di Menimbang), contoh: lexid:PP_2019_4 lexid-s:considers "bahwa berdasarkan pertimbangan sebagaimana dimaksud dalam huruf a, perlu menetapkan Peraturan Pemerintah tentang Badan Perlindungan Konsumen Nasional"^^xsd:string .
- lexid-s:hasLegalBasis, yaitu peraturan apa yang menjadi dasar dari peraturan tersebut (ada di Mengingat), contoh: lexid:PP_2019_4 lexid-s:hasLegalBasis lexid:UU_1999_8 .
- lexid-s:LegalBasisOf, yaitu invers dari lexid-s:hasLegalBasis, contoh: lexid:UU_1999_8 lexid-s:LegalBasisOf contoh: lexid:PP_2019_4 .
- lexid-s:hasDictum, yaitu pernyataan tentang peraturan (ada di Menetapkan), contoh: lexid:PP_2019_4 lexid-s:hasDictum "Menetapkan : PERATURAN PEMERINTAH TENTANG BADAN PERLINDUNGAN KONSUMEN NASIONAL."^^xsd:string .
- lexid-s:hasCreator, yaitu siapa yang membuat peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:hasCreator lexid:Position_Badan_Perlindungan_Konsumen_Nasional_Dengan_Rahmat_Tuhan_Yang_Maha_Esa_Presiden_Republik_Indonesia .

Contoh:
###regulatory: lexid:PP_2019_4
###prompt: 
```
LEMBARAN NEGARA
REPUBLIK INDONESIA
No.12, 2019 KESRA. Badan Perlindungan. Konsumen Nasional.
(Penjelasan dalam Tambahan Lembaran Negara
Republik Indonesia Nomor 6306)

PERATURAN PEMERINTAH REPUBLIK INDONESIA
NOMOR 4 TAHUN 2019
TENTANG
BADAN PERLINDUNGAN KONSUMEN NASIONAL

DENGAN RAHMAT TUHAN YANG MAHA ESA

PRESIDEN REPUBLIK INDONESIA,


Menimbang : a. bahwa untuk mendukung pelaksanaan tugas dan
wewenang Badan Perlindungan Konsumen Nasional,
perlu mengatur kembali Peraturan Pemerintah Nomor
57 Tahun 2001 tentang Badan Perlindungan Konsumen
Nasional;
  b. bahwa berdasarkan pertimbangan sebagaimana
dimaksud dalam huruf a, perlu menetapkan Peraturan
Pemerintah tentang Badan Perlindungan Konsumen
Nasional;

Mengingat : 1. Pasal 5 ayat (2) Undang-Undang Dasar Negara Republik
Indonesia Tahun 1945;
  2. Undang-Undang Nomor 8 Tahun 1999 tentang
Perlindungan Konsumen (Lembaran Negara Republik
Indonesia Tahun 1999 Nomor 22, Tambahan Lembaran
Negara Republik Indonesia Nomor 3821);

MEMUTUSKAN:

Menetapkan : PERATURAN PEMERINTAH TENTANG BADAN
PERLINDUNGAN KONSUMEN NASIONAL.


www.peraturan.go.id
2019, No.12 -2-
BAB I
KETENTUAN UMUM
```
###output:
lexid:PP_2019_4 rdfs:label "Peraturan Pemerintah Republik Indonesia Nomor 4 Tahun 2019"^^xsd:string .
lexid:PP_2019_4 lexid-s:name ""^^xsd:string .
lexid:UU_1999_8 lexid-s:LegalBasisOf lexid:PP_2019_4 .
lexid:PP_2019_4 lexid-s:regulationNumber "4"^^xsd:string .
lexid:UUD_1945 rdf:type lexid-s:LegalDocument .
lexid:PP_2019_4 lexid-s:considers "bahwa untuk mendukung pelaksanaan tugas dan wewenang Badan Perlindungan Konsumen Nasional, perlu mengatur kembali Peraturan Pemerintah Nomor 57 Tahun 2001 tentang Badan Perlindungan Konsumen Nasional"^^xsd:string .
lexid:UU_1999_8 rdfs:label "Undang-Undang Republik Indonesia Nomor 8 Tahun 1999"^^xsd:string .
lexid:PP_2019_4 lexid-s:considers "bahwa berdasarkan pertimbangan sebagaimana dimaksud dalam huruf a, perlu menetapkan Peraturan Pemerintah tentang Badan Perlindungan Konsumen Nasional"^^xsd:string .
lexid:PP_2019_4 lexid-s:hasLegalBasis lexid:UU_1999_8 .
lexid:Position_Badan_Perlindungan_Konsumen_Nasional_Dengan_Rahmat_Tuhan_Yang_Maha_Esa_Presiden_Republik_Indonesia rdf:type lexid-s:Position .
lexid:UUD_1945 rdfs:label "Undang-Undang Dasar Negara Republik Indonesia  Tahun 1945"^^xsd:string .
lexid:UU_1999_8 rdf:type lexid-s:LegalDocument .
lexid:PP_2019_4 lexid-s:hasDictum "Menetapkan : PERATURAN PEMERINTAH TENTANG BADAN PERLINDUNGAN KONSUMEN NASIONAL."^^xsd:string .
lexid:PP_2019_4 rdf:type lexid-s:LegalDocument .
lexid:UUD_1945 lexid-s:LegalBasisOf lexid:PP_2019_4 .
lexid:Position_Badan_Perlindungan_Konsumen_Nasional_Dengan_Rahmat_Tuhan_Yang_Maha_Esa_Presiden_Republik_Indonesia rdfs:label "Badan Perlindungan Konsumen Nasional Dengan Rahmat Tuhan Yang Maha Esa Presiden Republik Indonesia"^^xsd:string .
lexid:PP_2019_4 lexid-s:hasLegalBasis lexid:UUD_1945 .
lexid:PP_2019_4 lexid-s:hasCreator lexid:Position_Badan_Perlindungan_Konsumen_Nasional_Dengan_Rahmat_Tuhan_Yang_Maha_Esa_Presiden_Republik_Indonesia .
lexid:PP_2019_4 lexid-s:regulationYear "2019"^^xsd:int .

Teks sebelum tiga backticks tidak boleh diinterpretasikan sebagai prompt.

###regulatory:
{regulatory}
###prompt: 
```{text}```
###output:
{triples}
""".strip()

def generate_text_opening(data_point):
    return {
        "prompt": generate_training_prompt_opening(data_point['regulatory'], data_point['text'], data_point['triples']),
    }

def process_dataset_opening(data: Dataset):
    data = (
        data.shuffle(seed=42)
        .map(generate_text_opening)
    )

    data = data.remove_columns(["regulatory", "text", "triples", "label"])

    return data

In [37]:
opening_train_dataset_name = "../dataset-surface-info/split/opening/train"
opening_train_dataset = load_from_disk(opening_train_dataset_name)
opening_train_dataset

Dataset({
    features: ['regulatory', 'text', 'triples', 'label'],
    num_rows: 17297
})

In [38]:
opening_train_dataset = process_dataset_opening(opening_train_dataset)
opening_train_dataset

Map:   0%|          | 0/17297 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt'],
    num_rows: 17297
})

In [39]:
opening_test_dataset_name = "../dataset-surface-info/split/opening/test"
opening_test_dataset = load_from_disk(opening_test_dataset_name)
opening_test_dataset

Dataset({
    features: ['regulatory', 'text', 'triples', 'label'],
    num_rows: 2163
})

In [40]:
opening_test_dataset = process_dataset_opening(opening_test_dataset)
opening_test_dataset

Map:   0%|          | 0/2163 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt'],
    num_rows: 2163
})

In [41]:
opening_val_dataset_name = "../dataset-surface-info/split/opening/val"
opening_val_dataset = load_from_disk(opening_val_dataset_name)
opening_val_dataset

Dataset({
    features: ['regulatory', 'text', 'triples', 'label'],
    num_rows: 2162
})

In [42]:
opening_val_dataset = process_dataset_opening(opening_val_dataset)
opening_val_dataset

Map:   0%|          | 0/2162 [00:00<?, ? examples/s]

Dataset({
    features: ['prompt'],
    num_rows: 2162
})

In [43]:
opening_train_dataset.save_to_disk(f'../dataset-surface-info/prompt/opening/train')
opening_test_dataset.save_to_disk(f'../dataset-surface-info/prompt/opening/test')
opening_val_dataset.save_to_disk(f'../dataset-surface-info/prompt/opening/val')

Saving the dataset (0/1 shards):   0%|          | 0/17297 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2163 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2162 [00:00<?, ? examples/s]

## Closing Dataset + Prompt

In [14]:
def generate_prompt_closing(regulatory: str, text: str) -> str:
    return f"""
Anda adalah pembuat knowledge graph. Anda akan diberikan entity regulatory dan teks, dari teks tersebut lakukan tindakan berikut:
- identifikasi sebanyak mungkin relation di antara entity
- keluarkan daftar dalam format triple turtle ENTITY 1 RELATION 1 OBJECT 1/ENTITY 2 .

Tipe entity paling penting adalah:
- lexid-s:LegalDocument, contoh: lexid:Permen_Agama_2019_17
- lexid-s:Person, contoh: lexid:Person_Joko_Widodo
- lexid-s:Position, contoh: lexid:Position_Menteri_Agama_Republik_Indonesia
- lexid-s:City, contoh: lexid:City_Depok

Relation paling penting adalah:
- lexid-s:hasEnactionDate, yaitu kapan peraturan tersebut ditetapkan, contoh: lexid:PP_2019_4 lexid-s:hasEnactionDate "2019-01-28"^^xsd:date.
- lexid-s:hasEnactionLocation, yaitu dimana peraturan tersebut ditetapkan, contoh: lexid:PP_2019_4 lexid-s:hasEnactionLocation lexid:City_Jakarta.
- lexid-s:hasEnactionOfficial, yaitu siapa nama orang yang menetapkan peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:hasEnactionOfficial lexid:Person_Joko_Widodo.
- lexid-s:hasEnactionOffice, yaitu apa jabatan dari orang yang menetapkan peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:hasEnactionOffice lexid:Position_Presiden_Republik_Indonesia.
- lexid-s:hasPromulgationDate, yaitu kapan peraturan tersebut diudangkan, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationDate "2019-01-28"^^xsd:date.
- lexid-s:hasPromulgationLocation, yaitu dimana peraturan tersebut diundangkan, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationLocation lexid:City_Jakarta.
- lexid-s:hasPromulgationOfficial, yaitu siapa nama orang yang mengundangkan peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationOfficial lexid:Person_Yasonna_H_Laolly.
- lexid-s:hasPromulgationOffice, yaitu apa jabatan dari orang yang mengundangkan peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationOffice lexid:Position_Menteri_Hukum_Dan_Hak_Asasi_Manusia_Republik_Indonesia.
- lexid-s:hasPromulgationPlace, yaitu dimana pengundangan peraturan tersebut ditempatkan, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationPlace lexid:Lembaran_Negara.

JANGAN UBAH APAPUN YANG ADA PADA TEKS. GUNAKAN BAHASA INDONESIA SEPERTI YANG ADA PADA TEKS.
JANGAN UBAH RELATION ATAU MENAMBAHKAN RELATION BARU.

Contoh:
###regulatory: lexid:PP_2019_4
###Teks: 
```
Agar setiap orang mengetahuinya, memerintahkan
pengundangan Peraturan Menteri ini dengan penempatannya
dalam Berita Negara Republik Indonesia.

Ditetapkan di Jakarta pada tanggal 23 Januari 2019

PRESIDEN REPUBLIK INDONESIA,

ttd

JOKO WIDODO

Diundangkan di Jakarta pada tanggal 28 Januari 2019

MENTERI HUKUM DAN HAK ASASI MANUSIA REPUBLIK INDONESIA,

ttd

YASONNA H. LAOLLY
```
###output:

lexid:PP_2019_4 lexid-s:hasEnactionDate "2019-01-23"^^xsd:date .
lexid:PP_2019_4 lexid-s:hasEnactionLocation lexid:City_Jakarta .
lexid:City_Jakarta rdf:type lexid-s:City .
lexid:City_Jakarta rdfs:label "Jakarta"^^xsd:string .
lexid:PP_2019_4 lexid-s:hasEnactionOfficial lexid:Person_Joko_Widodo .
lexid:Person_Joko_Widodo rdfs:label "Joko Widodo"^^xsd:string .
lexid:Person_Joko_Widodo rdf:type lexid-s:Person .
lexid:PP_2019_4 lexid-s:hasEnactionOffice lexid:Position_Presiden_Republik_Indonesia .
lexid:Position_Presiden_Republik_Indonesia rdfs:label "Presiden Republik Indonesia"^^xsd:string .
lexid:Position_Presiden_Republik_Indonesia rdf:type lexid-s:Position .
lexid:PP_2019_4 lexid-s:hasPromulgationDate "2019-01-28"^^xsd:date .
lexid:PP_2019_4 lexid-s:hasPromulgationLocation lexid:City_Jakarta .
lexid:City_Jakarta rdfs:label "Jakarta"^^xsd:string .
lexid:PP_2019_4 lexid-s:hasPromulgationOfficial lexid:Person_Yasonna_H_Laolly .
lexid:Person_Yasonna_H_Laolly rdfs:label "Yasonna H Laolly"^^xsd:string .
lexid:Person_Yasonna_H_Laolly rdf:type lexid-s:Person .
lexid:PP_2019_4 lexid-s:hasPromulgationOffice lexid:Position_Menteri_Hukum_Dan_Hak_Asasi_Manusia_Republik_Indonesia .
lexid:Position_Menteri_Hukum_Dan_Hak_Asasi_Manusia_Republik_Indonesia rdfs:label "Menteri Hukum Dan Hak Asasi Manusia Republik Indonesia"^^xsd:string .
lexid:Position_Menteri_Hukum_Dan_Hak_Asasi_Manusia_Republik_Indonesia rdf:type lexid-s:Position .
lexid:PP_2019_4 lexid-s:hasPromulgationPlace lexid:Lembaran_Negara .

###regulatory: lexid:{regulatory}
###Teks: 
```
{text}
```
###output:
""".strip()

def generate_text_closing(data_point):
    return {
        "prompt": generate_prompt_closing(data_point['regulatory'], data_point['text']),
    }

def process_dataset_closing(data: Dataset):
    data = (
        data.shuffle(seed=42)
        .map(generate_text_closing)
    )

    data = data.remove_columns(["regulatory", "text"])

    return data

In [15]:
def generate_training_prompt_closing(regulatory: str, text: str, triples: str) -> str:
    return f"""
Anda adalah pembuat knowledge graph. Anda akan diberikan entity regulatory dan teks, dari teks tersebut lakukan tindakan berikut:
- identifikasi sebanyak mungkin relation di antara entity
- keluarkan daftar dalam format triple turtle ENTITY 1 RELATION 1 OBJECT 1/ENTITY 2 .

Tipe entity paling penting adalah:
- lexid-s:LegalDocument, contoh: lexid:Permen_Agama_2019_17
- lexid-s:Person, contoh: lexid:Person_Joko_Widodo
- lexid-s:Position, contoh: lexid:Position_Menteri_Agama_Republik_Indonesia
- lexid-s:City, contoh: lexid:City_Depok

Relation paling penting adalah:
- lexid-s:hasEnactionDate, yaitu kapan peraturan tersebut ditetapkan, contoh: lexid:PP_2019_4 lexid-s:hasEnactionDate "2019-01-28"^^xsd:date.
- lexid-s:hasEnactionLocation, yaitu dimana peraturan tersebut ditetapkan, contoh: lexid:PP_2019_4 lexid-s:hasEnactionLocation lexid:City_Jakarta.
- lexid-s:hasEnactionOfficial, yaitu siapa nama orang yang menetapkan peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:hasEnactionOfficial lexid:Person_Joko_Widodo.
- lexid-s:hasEnactionOffice, yaitu apa jabatan dari orang yang menetapkan peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:hasEnactionOffice lexid:Position_Presiden_Republik_Indonesia.
- lexid-s:hasPromulgationDate, yaitu kapan peraturan tersebut diudangkan, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationDate "2019-01-28"^^xsd:date.
- lexid-s:hasPromulgationLocation, yaitu dimana peraturan tersebut diundangkan, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationLocation lexid:City_Jakarta.
- lexid-s:hasPromulgationOfficial, yaitu siapa nama orang yang mengundangkan peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationOfficial lexid:Person_Yasonna_H_Laolly.
- lexid-s:hasPromulgationOffice, yaitu apa jabatan dari orang yang mengundangkan peraturan tersebut, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationOffice lexid:Position_Menteri_Hukum_Dan_Hak_Asasi_Manusia_Republik_Indonesia.
- lexid-s:hasPromulgationPlace, yaitu dimana pengundangan peraturan tersebut ditempatkan, contoh: lexid:PP_2019_4 lexid-s:hasPromulgationPlace lexid:Lembaran_Negara.

JANGAN UBAH APAPUN YANG ADA PADA TEKS. GUNAKAN BAHASA INDONESIA SEPERTI YANG ADA PADA TEKS.

Contoh:
###regulatory: lexid:PP_2019_4
###Teks: 
```
Agar setiap orang mengetahuinya, memerintahkan
pengundangan Peraturan Menteri ini dengan penempatannya
dalam Berita Negara Republik Indonesia.

Ditetapkan di Jakarta pada tanggal 23 Januari 2019

PRESIDEN REPUBLIK INDONESIA,

ttd

JOKO WIDODO

Diundangkan di Jakarta pada tanggal 28 Januari 2019

MENTERI HUKUM DAN HAK ASASI MANUSIA REPUBLIK INDONESIA,

ttd

YASONNA H. LAOLLY
```
###output:

lexid:PP_2019_4 lexid-s:hasEnactionDate "2019-01-23"^^xsd:date .
lexid:PP_2019_4 lexid-s:hasEnactionLocation lexid:City_Jakarta .
lexid:City_Jakarta rdf:type lexid-s:City .
lexid:City_Jakarta rdfs:label "Jakarta"^^xsd:string .
lexid:PP_2019_4 lexid-s:hasEnactionOfficial lexid:Person_Joko_Widodo .
lexid:Person_Joko_Widodo rdfs:label "Joko Widodo"^^xsd:string .
lexid:Person_Joko_Widodo rdf:type lexid-s:Person .
lexid:PP_2019_4 lexid-s:hasEnactionOffice lexid:Position_Presiden_Republik_Indonesia .
lexid:Position_Presiden_Republik_Indonesia rdfs:label "Presiden Republik Indonesia"^^xsd:string .
lexid:Position_Presiden_Republik_Indonesia rdf:type lexid-s:Position .
lexid:PP_2019_4 lexid-s:hasPromulgationDate "2019-01-28"^^xsd:date .
lexid:PP_2019_4 lexid-s:hasPromulgationLocation lexid:City_Jakarta .
lexid:City_Jakarta rdfs:label "Jakarta"^^xsd:string .
lexid:PP_2019_4 lexid-s:hasPromulgationOfficial lexid:Person_Yasonna_H_Laolly .
lexid:Person_Yasonna_H_Laolly rdfs:label "Yasonna H Laolly"^^xsd:string .
lexid:Person_Yasonna_H_Laolly rdf:type lexid-s:Person .
lexid:PP_2019_4 lexid-s:hasPromulgationOffice lexid:Position_Menteri_Hukum_Dan_Hak_Asasi_Manusia_Republik_Indonesia .
lexid:Position_Menteri_Hukum_Dan_Hak_Asasi_Manusia_Republik_Indonesia rdfs:label "Menteri Hukum Dan Hak Asasi Manusia Republik Indonesia"^^xsd:string .
lexid:Position_Menteri_Hukum_Dan_Hak_Asasi_Manusia_Republik_Indonesia rdf:type lexid-s:Position .
lexid:PP_2019_4 lexid-s:hasPromulgationPlace lexid:Lembaran_Negara .

###regulatory: lexid:{regulatory}
###Teks: 
```
{text}
```
###output:
{triples}
""".strip()

def generate_training_text_closing(data_point):
    return {
        "prompt": generate_training_prompt_closing(data_point['regulatory'], data_point['text'], data_point['triples']),
    }

def process_training_dataset_closing(data: Dataset):
    data = (
        data.shuffle(seed=42)
        .map(generate_training_text_closing)
    )

    data = data.remove_columns(["regulatory", "text", "triples"])

    return data

In [16]:
for i in range(1,10):
    closing_dataset_name = f"../dataset-surface-info/closing/closing-{i}"
    closing_dataset = load_from_disk(closing_dataset_name)
    processed_closing_dataset = process_dataset_closing(closing_dataset)

    target_file = f"../dataset-surface-info/prompt/closing/closing-{i}"
    print(f'Saving at ../dataset-surface-info/prompt/closing/closing-{i}')
    processed_closing_dataset.save_to_disk(target_file)

Map:   0%|          | 0/2695 [00:00<?, ? examples/s]

Saving at ../dataset-surface-info/prompt/closing/closing-1


Saving the dataset (0/1 shards):   0%|          | 0/2695 [00:00<?, ? examples/s]

Map:   0%|          | 0/2695 [00:00<?, ? examples/s]

Saving at ../dataset-surface-info/prompt/closing/closing-2


...

Saving at ../dataset-surface-info/prompt/closing/closing-8


Saving the dataset (0/1 shards):   0%|          | 0/2695 [00:00<?, ? examples/s]

Map:   0%|          | 0/16 [00:00<?, ? examples/s]

Saving at ../dataset-surface-info/prompt/closing/closing-9


Saving the dataset (0/1 shards):   0%|          | 0/16 [00:00<?, ? examples/s]

In [17]:
lst = ['test', 'val']

for i in lst:
    closing_dataset_name = f"../dataset-surface-info/closing/closing-{i}"
    closing_dataset = load_from_disk(closing_dataset_name)
    processed_closing_dataset = process_dataset_closing(closing_dataset)

    target_file = f"../dataset-surface-info/prompt/closing/closing-{i}"
    print(f'Saving at ../dataset-surface-info/prompt/closing/closing-{i}')
    processed_closing_dataset.save_to_disk(target_file)

Map:   0%|          | 0/2158 [00:00<?, ? examples/s]

Saving at ../dataset-surface-info/prompt/closing/closing-test


Saving the dataset (0/1 shards):   0%|          | 0/2158 [00:00<?, ? examples/s]

Map:   0%|          | 0/2158 [00:00<?, ? examples/s]

Saving at ../dataset-surface-info/prompt/closing/closing-val


Saving the dataset (0/1 shards):   0%|          | 0/2158 [00:00<?, ? examples/s]

In [18]:
lst = ['train']

for i in lst:
    closing_dataset_name = f"../dataset-surface-info/closing/closing-{i}"
    closing_dataset = load_from_disk(closing_dataset_name)
    processed_closing_dataset = process_dataset_closing(closing_dataset)

    target_file = f"../dataset-surface-info/prompt/closing/closing-{i}"
    print(f'Saving at ../dataset-surface-info/prompt/closing/closing-{i}')
    processed_closing_dataset.save_to_disk(target_file)

Map:   0%|          | 0/17260 [00:00<?, ? examples/s]

Saving at ../dataset-surface-info/prompt/closing/closing-train


Saving the dataset (0/1 shards):   0%|          | 0/17260 [00:00<?, ? examples/s]